# With ChatModel

In [ ]:
import json
import helper
import pandas as pd

from datetime import datetime

from langchain.chat_models import ChatOpenAI
from langchain.prompts import load_prompt
from langchain.schema import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
def save_result(output, prompt_type, model, init, best_practice, format_prompt, output_parser, ticketUri, ticket, reruns=0, evalMode=False):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    result = {}
    result['model'] = model
    result['creation_timestamp'] = date + ' ' + time
    result['prompts'] = {
        'init_uri': init,
        'best_practice_uri': best_practice,
        'formatter_uri': format_prompt}
    result['output_parser'] = output_parser
    result['input_data'] = {
        'ticket_uri': ticketUri,
        'jira': ticket['Jira'],
        'id': ticket['IssueId'],
        'evolution': ticket['EvoId']}
    result['reruns'] = reruns
    result['output'] = output

    helper.annotateResult(result, ticket, prompt_type, evalMode)

    if evalMode:
        with open('evaluation/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
            json.dump(result, f, indent=4)
        print('Evaluation mode: result saved in evaluation folder')

    with open('results/'+prompt_type+'/output_'+date+'_'+time+'.json', 'w') as f:
        json.dump(result, f, indent=4)

In [ ]:
model_name = 'gpt-4'

model = ChatOpenAI(
    model=model_name,
    api_key=open('api.txt', 'r').read(),
    temperature=0
)

## Summary Length

#### Detection

In [ ]:
def runDetectionOfSummaryLength(init_prompt_name, bestPractice_prompt_name, format_prompt_name, ticketUri, ticket, reruns= 0, evalMode=False):
    ### Set variables ###
    minChars = 39
    maxChars = 70

    init_prompt = load_prompt('prompts/init/'+init_prompt_name)
    bestPractice_prompt = load_prompt('prompts/summary/'+bestPractice_prompt_name).format(min=minChars, max=maxChars)
    format_prompt = load_prompt('prompts/summary/'+format_prompt_name)

    ### Create multi chain ###
    chain1 = init_prompt | model | StrOutputParser()
    chain2 = (
        {"revised_ticket": chain1}
        | format_prompt
        | model
        | JsonOutputParser()
    )

    ### Run chains ###
    output = chain2.invoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":ticket})

    ### Save result ###
    save_result(output, 'summary', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', ticketUri, ticket, reruns, evalMode)

In [ ]:
annotatedDataset = pd.read_csv('data/summary/summaryDataset.csv')

testDataset = annotatedDataset.sample(2)
testDataset['violation_predicted'] = None

for index, row in annotatedDataset.iterrows():
    ticketUri = "./data/summary/" + row['Jira'] + "_" + str(row['IssueId']) + "_" + str(row['EvoId']) + ".json"

    # Set preditedc value as new entry in the dataset
    # row['violation_predicted'] = "TRUE"


In [ ]:
# runDetectionOfSummaryLength('initPrompt_V1.3.0.json', 'summaryLengthPrompt_V2.3.0.json', 'summaryLengthFormatPrompt_V1.2.1.json', json_sample. ticketUri)

## Description Completeness

In [ ]:
sample_name = "suggestion_sample.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'descriptionCompletenessPrompt_V1.0.0.json'
bestPractice_prompt = load_prompt('prompts/description/'+bestPractice_prompt_name).format()

format_prompt_name = 'descriptionCompletenessFormatPrompt_V1.0.0.json'
format_prompt = load_prompt('prompts/description/'+format_prompt_name)

In [ ]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [ ]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

In [ ]:
save_result(output, 'description', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name)

## Field Update

maybe try Few-Shot here for better results??

In [ ]:
sample_name = "suggestion_sample.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'updatePrompt_V1.16.0.json'
bestPractice_prompt = load_prompt('prompts/update/'+bestPractice_prompt_name).format()

format_prompt_name = 'updateFormatPrompt_V1.3.0.json'
format_prompt = load_prompt('prompts/update/'+format_prompt_name)

In [ ]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [ ]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})

In [ ]:
save_result(output, 'update', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name)

## Toxic Speech Detection

In [ ]:
sample_name = "last_row_toxic.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'toxicSpeechPrompt_V1.5.0.json'
bestPractice_prompt = load_prompt('prompts/toxicSpeech/'+bestPractice_prompt_name).format()

format_prompt_name = 'toxicSpeechFormatPrompt_V1.2.0.json'
format_prompt = load_prompt('prompts/toxicSpeech/'+format_prompt_name)

In [ ]:
chain1 = init_prompt | model | StrOutputParser()
chain2 = (
    {"revised_ticket": chain1}
    | format_prompt
    | model
    | JsonOutputParser()
)

In [ ]:
output = await chain2.ainvoke({"role":"Content Moderator", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

In [ ]:
save_result(output, 'toxicSpeech', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name)

## Bug Report Structure

In [ ]:
sample_name = "desc_struc_last_row.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
bugReportStrucutre_prompt_name = 'bugReportStructurePrompt_V2.3.0.json'
bugReportStrucutre_prompt = load_prompt('prompts/bugReportStructure/'+bugReportStrucutre_prompt_name)

In [ ]:
chain = bugReportStrucutre_prompt | model | StrOutputParser()

In [ ]:
output = chain.invoke({"bug_report":json_sample})

Why data from comments field is taken into account here?

In [ ]:
output = output.replace('\\n', '\n').replace('\\t', '\t')
print(output)


In [ ]:
save_result(output, 'bugReportStructure', model_name, '-', bugReportStrucutre_prompt_name, '-', 'StrOutputParser', sample_name)

## Internationalization

In [ ]:
sample_name = "last_row_german.json"
with open('./data/json/last_row/'+sample_name) as f:
    json_sample = json.load(f)

In [ ]:
init_prompt_name = 'initPrompt_V1.3.0.json'
init_prompt = load_prompt('prompts/init/'+init_prompt_name)

bestPractice_prompt_name = 'internationalizationPrompt_V1.5.0.json'
bestPractice_prompt = load_prompt('prompts/internationalization/'+bestPractice_prompt_name).format()

format_prompt_name = 'internationalizationFormatPrompt_V1.2.0.json'
format_prompt = load_prompt('prompts/internationalization/'+format_prompt_name)

In [ ]:
chain1 = init_prompt | model | StrOutputParser()

chain2 = (
    {"revised_ticket":chain1}
    | format_prompt
    | model
    | JsonOutputParser()
    )

In [ ]:
output = await chain2.ainvoke({"role":"Software Engineer", "best_practice":bestPractice_prompt, "ticket":json_sample})
output

In [ ]:
save_result(output, 'internationalization', model_name, init_prompt_name, bestPractice_prompt_name, format_prompt_name, 'JsonOutputParser', sample_name)